# 🤖 Operation Ledger-Mind: Fine-Tuning on Google Colab

This notebook fine-tunes Llama-3 8B on your Q&A dataset using a **FREE** T4 GPU.

## Setup Instructions:
1. Go to **Runtime → Change runtime type → T4 GPU**
2. Upload your dataset files to Colab
3. Run all cells
4. Download the trained model adapters

**Expected Time**: 1.5-2 hours

In [1]:
# 1. Install dependencies
!pip install -q transformers datasets peft bitsandbytes accelerate trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 14.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 23.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 41.9 MB/s eta 0:00:00


In [2]:
# 2. Check GPU
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
# 3. Upload your Hugging Face token
from google.colab import userdata
import os

# Option 1: Set as Colab secret (Recommended)
# Go to: 🔑 (left sidebar) → Add HF_TOKEN
try:
    HF_TOKEN = userdata.get('HF_TOKEN')
except:
    # Option 2: Paste directly (less secure)
    HF_TOKEN = input("Enter your Hugging Face token: ")

os.environ['HF_TOKEN'] = HF_TOKEN
print("✓ Token set")

✓ Token set


In [5]:
# 4. Upload dataset files
from google.colab import files
import os

print("Upload train.jsonl and golden_test_set.jsonl from your datasets/ folder")

os.makedirs('datasets', exist_ok=True)
uploaded = files.upload()

# Move files to datasets folder
for filename in uploaded.keys():
    !mv {filename} datasets/{filename}

print("\n✓ Dataset files uploaded")

Upload train.jsonl and golden_test_set.jsonl from your datasets/ folder


KeyboardInterrupt: 

In [ ]:
# 5. Import libraries
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
from trl import SFTTrainer

print("✓ Libraries imported")

In [ ]:
# 6. Load model with 4-bit quantization
model_name = "meta-llama/Meta-Llama-3-8B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print(f"Loading {model_name}...")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    token=HF_TOKEN
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    token=HF_TOKEN
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = prepare_model_for_kbit_training(model)

print("✓ Model loaded with 4-bit quantization")

In [ ]:
# 7. Configure LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

print("✓ LoRA configured")

In [ ]:
# 8. Load dataset
dataset = load_dataset('json', data_files={
    'train': 'datasets/train.jsonl',
    'test': 'datasets/golden_test_set.jsonl'
})

print(f"✓ Loaded {len(dataset['train'])} training samples")
print(f"✓ Loaded {len(dataset['test'])} test samples")

In [ ]:
# 9. Format instruction function
def format_instruction(sample):
    return f"""### Instruction:
{sample['instruction']}

### Response:
{sample['output']}"""

In [ ]:
# 10. Configure training
training_args = TrainingArguments(
    output_dir="./lora_adapters",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    warmup_steps=50,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    report_to="none"
)

print("✓ Training configured")

In [ ]:
# 11. Create trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    peft_config=lora_config,
    dataset_text_field="text",
    formatting_func=format_instruction,
    tokenizer=tokenizer,
    args=training_args,
    max_seq_length=512,
)

print("✓ Trainer created")

In [ ]:
# 12. START TRAINING! 🚀
print("="*60)
print("TRAINING STARTED")
print("="*60)
print("This will take ~1.5-2 hours on T4 GPU")
print("You can close this tab - training will continue!")
print("="*60)

trainer.train()

In [ ]:
# 13. Save model
trainer.save_model("./lora_adapters")
print("\n✓ Model saved to lora_adapters/")

In [ ]:
# 14. Test the model
test_question = "What was Uber's revenue in 2024?"

prompt = f"""### Instruction:
{test_question}

### Response:
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.7,
        do_sample=True
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\nTest Question:", test_question)
print("\nModel Answer:")
print(response.split("### Response:")[1].strip())

In [ ]:
# 15. Download the trained model
!zip -r lora_adapters.zip lora_adapters/

from google.colab import files
files.download('lora_adapters.zip')

print("\n✓ Download started!")
print("\nExtract this on your local PC to: models/lora_adapters/")

## 🎉 Done!

### Next Steps:
1. Download `lora_adapters.zip` (should start automatically)
2. Extract to your local PC: `Mini Project 1/models/lora_adapters/`
3. Run evaluation: `python utils/evaluate_systems.py`
4. Launch web UI: `python web_ui.py`

**Training Complete!** ✅